In [1]:
import sys
sys.path.append("../src")
import matplotlib.pyplot as plt
import pickle
import torch
from torch_geometric.data import Data

%load_ext autoreload
%autoreload 2

from data import *
from loader import *
from visualize import plot_merger_tree

# Prepare data

In [2]:
# from data.py
subhalos = prepare_subhalos()
len(subhalos)

/home/john/miniforge3/envs/pyg/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/john/miniforge3/envs/pyg/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/john/miniforge3/envs/pyg/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/john/miniforge3/envs/pyg/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/john/miniforge3/envs/pyg/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/john/miniforge3/env

132953

In [5]:
subhalos.to_parquet("../results/subhalos.parquet")

## Env

In [7]:
# from loader.py
env_data = make_cosmic_graph(subhalos, 3) 
env_data

/home/john/research/env-merger-gnns/notebook/../src/loader.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  index=torch.tensor(target_nodes),


Data(x=[132953, 3], edge_index=[2, 6653247], edge_attr=[6653247, 6], y=[132953, 2], pos=[132953, 3], vel=[132953, 3], is_central=[132953, 1], x_hydro=[132953, 2], pos_hydro=[132953, 3], vel_hydro=[132953, 3], halfmassradius=[132953, 1], subhalo_id=[132953], overdensity=[132953])

In [8]:
with open("../results/cosmic_graphs_3Mpc.pkl", "wb") as f:
    pickle.dump(env_data, f)

## Trees

In [9]:
# from data -- takes like 5 min (and 40+GB of memory)
all_trees = load_trees()


In [10]:
# from loader.py
tree_data = make_merger_tree_graphs(all_trees, subhalos)
len(tree_data)

41575 0 9952


123004

N_not_in_subhalo_cat = 5396

N_is_satellite = 0

N_bad_stellar_mass = 5437

In [11]:
with open("../results/merger_trees.pkl", "wb") as f:
    pickle.dump(tree_data, f)

# Reload all

In [2]:
subhalos = pd.read_parquet("../results/subhalos.parquet")

with open("../results/cosmic_graphs_3Mpc.pkl", "rb") as f:
    env_data = pickle.load(f)

with open("../results/merger_trees.pkl", "rb") as f:
    tree_data = pickle.load(f)

# Visualize data

## Double check stellar mass--halo mass relations

In [12]:
plt.figure(figsize=(4,4), dpi=150)

plt.scatter(env_data.x[:, 0], env_data.y[:, 0], s=1, edgecolors="none", label="Env SHMR")
plt.scatter(*list(zip(*[(tree.x[0][0].item(), tree.y[0].item()) for tree in tree_data])), s=1, edgecolors="none", label="Mergers SHMR")
plt.legend(markerscale=8)
plt.xlabel(r"log($M_{\rm subhalo}/M_{\odot}$)", fontsize=14)
plt.ylabel(r"log($M_{\bigstar}/M_{\odot}$)", fontsize=14)
plt.grid(alpha=0.15)

plt.text(0.95, 0.2, "Centrals", color="C3", fontsize=16, transform=plt.gca().transAxes, ha="right")
plt.text(0.95, 0.1, "Satellites", color="C0", fontsize=16, transform=plt.gca().transAxes, ha="right")

plt.clf()

<Figure size 600x600 with 0 Axes>

The point of the above: these are exactly on top of each other.

In [13]:
plt.figure(figsize=(4,4), dpi=300)

plt.scatter(
    env_data.x[:, 0].numpy(), 
    env_data.y[:, 0].numpy(), 
    c=np.where(env_data.is_central.numpy().flatten(), "C3", "C0"), 
    s=1,
    edgecolors="none",
    rasterized=True
)
plt.xlabel(r"log($M_{\rm subhalo}/M_{\odot}$)", fontsize=14)
plt.ylabel(r"log($M_{\bigstar}/M_{\odot}$)", fontsize=14)
plt.grid(alpha=0.15)

plt.text(0.95, 0.2, "Centrals", color="C3", fontsize=16, transform=plt.gca().transAxes, ha="right")
plt.text(0.95, 0.1, "Satellites", color="C0", fontsize=16, transform=plt.gca().transAxes, ha="right")

plt.savefig("../results/figures/smhr.pdf")

plt.clf()

<Figure size 1200x1200 with 0 Axes>

In [14]:
# subhalo_id, log stellar mass
env_dict = {k.item():v.item() for k, v in zip(env_data.subhalo_id, env_data.x[:, 0])}
env_dict

{0: 14.62812328338623,
 1: 13.686690330505371,
 4: 12.746826171875,
 3: 12.747556686401367,
 5: 12.731315612792969,
 6: 12.403141021728516,
 9: 12.35177993774414,
 12: 12.312187194824219,
 11: 12.320868492126465,
 10: 12.330202102661133,
 13: 12.255717277526855,
 8: 12.364133834838867,
 15: 12.054425239562988,
 18: 11.930941581726074,
 17: 11.962911605834961,
 14: 12.090988159179688,
 24: 11.776555061340332,
 22: 11.8333740234375,
 21: 11.87842082977295,
 23: 11.804162979125977,
 26: 11.75743579864502,
 30: 11.63388442993164,
 42: 11.519027709960938,
 25: 11.77331256866455,
 45: 11.471420288085938,
 32: 11.60808277130127,
 28: 11.685644149780273,
 31: 11.632532119750977,
 39: 11.541378021240234,
 29: 11.648955345153809,
 27: 11.72299575805664,
 35: 11.573285102844238,
 33: 11.600592613220215,
 34: 11.583480834960938,
 37: 11.561906814575195,
 51: 11.421018600463867,
 38: 11.544342041015625,
 52: 11.411579132080078,
 54: 11.402844429016113,
 43: 11.482260704040527,
 53: 11.4048023223876

In [15]:
merger_dict = {tree.root_subhalo_id: tree.x[0][0].item() for tree in tree_data}
merger_dict = {k: merger_dict[k] for k in sorted(merger_dict)}
merger_dict

{0: 14.62812328338623,
 1: 13.686690330505371,
 3: 12.747556686401367,
 4: 12.746826171875,
 5: 12.731315612792969,
 6: 12.403141021728516,
 8: 12.364133834838867,
 9: 12.35177993774414,
 10: 12.330202102661133,
 11: 12.320868492126465,
 12: 12.312187194824219,
 13: 12.255717277526855,
 14: 12.090988159179688,
 15: 12.054425239562988,
 17: 11.962911605834961,
 18: 11.930941581726074,
 20: 11.895853042602539,
 21: 11.87842082977295,
 22: 11.8333740234375,
 23: 11.804162979125977,
 24: 11.776555061340332,
 25: 11.77331256866455,
 26: 11.75743579864502,
 27: 11.72299575805664,
 28: 11.685644149780273,
 29: 11.648955345153809,
 30: 11.63388442993164,
 31: 11.632532119750977,
 32: 11.60808277130127,
 33: 11.600592613220215,
 34: 11.583480834960938,
 35: 11.573285102844238,
 36: 11.569258689880371,
 37: 11.561906814575195,
 38: 11.544342041015625,
 39: 11.541378021240234,
 40: 11.541156768798828,
 41: 11.522136688232422,
 42: 11.519027709960938,
 43: 11.482260704040527,
 44: 11.4725618362426

## Plot merger tree

In [48]:
tree = tree_data[100]

plot_merger_tree(
    tree, 
    save_figname=f"tree_{tree.root_subhalo_id}", 
    title=f"Root Subhalo ID: {tree.root_subhalo_id}",
    vmax=11
)
plt.clf();

<Figure size 1800x2400 with 0 Axes>

## Plot env point cloud

In [49]:
from mpl_toolkits.mplot3d import Axes3D


In [62]:
env_data

Data(x=[132953, 3], edge_index=[2, 6653247], edge_attr=[6653247, 6], y=[132953, 2], pos=[132953, 3], vel=[132953, 3], is_central=[132953, 1], x_hydro=[132953, 2], pos_hydro=[132953, 3], vel_hydro=[132953, 3], halfmassradius=[132953, 1], subhalo_id=[132953], overdensity=[132953])

In [ ]:
fig = plt.figure(figsize=(12, 10), dpi=150)
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter3D(
    *env_data.pos.T.numpy(), 
    c=env_data.y[:, 0].numpy().flatten(), 
    s=(env_data.y[:, 0].numpy().flatten() - 8) ** 3, 
    vmin=8.5, 
    vmax=11,
    cmap='magma',
    rasterized=True
)

cb = plt.colorbar(sc, aspect=50, shrink=0.8)
cb.set_label(r'log($M_{\rm subhalo}/M_{\odot}$)', fontsize=12)

ax.set_xlabel('X [Mpc]', fontsize=12)
ax.set_ylabel('Y [Mpc]', fontsize=12)
ax.set_zlabel('Z [Mpc]', fontsize=12)
ax.set_title('TNG Subhalos', fontsize=16)

plt.savefig("../results/figures/cosmic-graph.pdf")

plt.clf()